In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import timescaledb_model as tsdb
from analyze import get_files_infos_df, read_file, get_files_infos_windows_df
from tqdm import tqdm 
from utils import multi_read_df_from_paths
from constant import DATA_PATH
import os

In [ ]:
files_info_path = os.path.join(DATA_PATH, 'files_infos.pkl')
# files_infos_df = get_files_infos_df()
# files_infos_df = get_files_infos_windows_df()  # for windows
# files_infos_df.to_pickle(files_info_path)

## Files Dataframe - Filtering
1 hour for each Last day of Each month

In [ ]:
files_infos_df = get_files_infos_df(files_info_path)
# files_infos_df = get_files_infos_windows_df(files_info_path) #for windows

In [ ]:
np.random.seed(10)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]

In [ ]:
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["market"]]).first().reset_index()

# Utils

In [ ]:
import re

def is_possible_ticker_with_number(symbol):
    return bool(re.match(r'^[A-Z0-9\-]{1,10}$', symbol))

def is_possible_ticker(symbol):
    return bool(re.match(r'^[A-Z]{1,10}$', symbol))

def detection_intrus(df: pd.DataFrame):
    df["is_ticker"] = df["symbol"].apply(is_possible_ticker_with_number)
    df["is_ticker_just_letters"] = df["symbol"].apply(is_possible_ticker)
    return df[(df["is_ticker"] == False) | (df["is_ticker_just_letters"] == False)]

In [ ]:
def get_prefix(df: pd.DataFrame, prefix_function = lambda x : x[0:3]) -> set:
    df["prefix"] = df["symbol"].apply(prefix_function)
    return set(df["prefix"].unique())

In [ ]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:].split("_")[0] if len(x) != 15 else np.nan,  # EuroNext Pars
    "1rA": lambda x: x[3:],  # EuroNext Amsterdam
    "1rE": lambda x: x[4:],  # EuroNext Paris
    "FF1": lambda x: x.split("_")[1][0:],  # EuroNext Brussels
}

map_prefix_to_mid = {
    "1rP": lambda is_compA: 7 if is_compA else 8,  # EuroNext Pars
    "1rA": lambda _: 6,  # EuroNext Amsterdam
    "1rE": lambda is_compA: 7 if is_compA else 8,  # EuroNext Paris
    "FF1": lambda _: 10,  # EuroNext Brussels
}


def add_symbol_nf_column(
    df: pd.DataFrame,
) -> pd.DataFrame:
    df["symbol_nf"] = df.apply(
        lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: x["symbol"])(
            x["symbol"]
        ),
        axis=1,
    )
    return df


def add_mid_column(
    df: pd.DataFrame,
    mid_default: int,
    is_compA: bool,
) -> pd.DataFrame:
    df["mid"] = df.apply(
        lambda x: map_prefix_to_mid.get(x["prefix"], lambda _: mid_default)(is_compA),
        axis=1,
    )
    return df

In [ ]:
def dfs_to_companie(dfs: list[pd.DataFrame], default_mid: int, is_compA: bool = False) -> pd.DataFrame:
    df_all_days = pd.concat(dfs)
    df_all_days_grouped = df_all_days.reset_index(drop=True).groupby(["symbol", "name"]).count()
    df_all_days_grouped["name"] = df_all_days_grouped.index.get_level_values(1)
    df_companies = df_all_days_grouped.groupby(df_all_days_grouped.index.get_level_values(0)).last()
    df_companies = df_companies.reset_index()[["symbol", "name"]]
    df_companies["prefix"] = df_companies["symbol"].apply(lambda x: x[0:3])
    df_companies = add_symbol_nf_column(df_companies)
    df_companies = add_mid_column(df_companies, default_mid, is_compA)
    df_companies = df_companies.drop(columns=["prefix"])
    return df_companies

## Amsterdam - Companies handling

In [ ]:
dfs_amsterdam = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "amsterdam"] ["path"]))

In [ ]:
dfs_amsterdam[100]

In [ ]:
mid = 6 #TODO get mid from db
amsterdam_companies = dfs_to_companie(dfs_amsterdam, mid) 

In [68]:
amsterdam_companies

,symbol,name,symbol_nf,mid,is_ticker,is_ticker_just_letters
0,124718367,ASTRAZENECA FIN CVR,124718367,6,True,False
1,AABA,ALTABA,AABA,6,True,True
2,AACG,ATA CRTV GLB SP ADR,AACG,6,True,True
3,AACI,ARMADA ACQN I,AACI,6,True,True
4,AACIU,ARMADA ACQN I UTS,AACIU,6,True,True
...,...,...,...,...,...,...
6684,ZWRKU,Z-WORK ACQN UTS 26,ZWRKU,6,True,True
6685,ZY,ZYMERGEN,ZY,6,True,True
6686,ZYME,ZYMEWORKS,ZYME,6,True,True
6687,ZYNE,ZYNERBA PHARMA,ZYNE,6,True,True


In [16]:
detection_intrus(amsterdam_companies)

,symbol,name,symbol_nf,mid,is_ticker,is_ticker_just_letters
0,124718367,ASTRAZENECA FIN CVR,124718367,6,True,False
290,ALKS_V,ALKERMES-EX,ALKS_V,6,False,False
2410,GASS1,GASS1 PACKAGE,GASS1,6,True,False
2414,GB2814265,MAGMA DESIGN AUTO,GB2814265,6,True,False
2684,GSRM_R,GSR II METEORA RTS,GSRM_R,6,False,False
5375,SGBX_V,SAFE & GREEN RG-EX,SGBX_V,6,False,False
5379,SGDV_V,SAFE AND RG-WI,SGDV_V,6,False,False
5950,TK112803851,FRD FUT INTELL RG-A,TK112803851,6,False,False


## CompA - Companies handling

In [17]:
dfs_compA = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "compA"] ["path"]))

100%|██████████| 540/540 [00:02<00:00, 234.39it/s]


In [50]:
for df in dfs_compA:
    df["timestamp"] = df.attrs["timestamp"]
compA_df = pd.concat(dfs_compA)
compA_df = compA_df.reset_index(drop=True)

In [57]:
compA_df.groupby(["symbol", "name"]).first().loc["1rPABIO"]

,last,volume,last_suffix,timestamp
name,,,,
ALBIOMA,45.00,7838,None,2021-02-03 09:02:02
SRDALBIOMA,34.02,1494,s,2022-01-14 09:02:01


In [52]:
compA_df.groupby(["symbol", "name"]).last().loc["1rPABIO"]

,last,volume,last_suffix,timestamp
name,,,,
ALBIOMA,33.12,26028,None,2021-12-09 17:02:01
SRDALBIOMA,47.14,0,c,2023-12-21 17:02:02


In [18]:
dfs_compA[-1][dfs_compA[-1]["last_suffix"] == "s"]

,last,volume,symbol,name,last_suffix
symbol,,,,,
1rPRE,175.0,0,1rPRE,SRDCOLAS,s


In [19]:
mid = 7 #TODO get mid from db
compA_companies = dfs_to_companie(dfs_compA, 7, is_compA=True)

In [20]:
compA_companies

,symbol,name,symbol_nf,mid
0,1rAAF,AIR FRANCE - KLM,AF,6
1,1rAAFA,AIR FRANCE - KLM,AFA,6
2,1rAENX,EURONEXT,ENX,6
3,1rAGTO,GEMALTO,GTO,6
4,1rARDSADT,ROYAL D SH DRP 21,RDSADT,6
...,...,...,...,...
404,FF11_ENX,EURONEXT,ENX,10
405,FF11_FP,TOTALENERGIES,FP,10
406,FF11_SEV,SUEZ,SEV,10
407,FF11_SGO,SAINT-GOBAIN,SGO,10


In [21]:
detection_intrus(compA_companies)

,symbol,name,symbol_nf,mid,is_ticker,is_ticker_just_letters
0,1rAAF,AIR FRANCE - KLM,AF,6,False,False
1,1rAAFA,AIR FRANCE - KLM,AFA,6,False,False
2,1rAENX,EURONEXT,ENX,6,False,False
3,1rAGTO,GEMALTO,GTO,6,False,False
4,1rARDSADT,ROYAL D SH DRP 21,RDSADT,6,False,False
...,...,...,...,...,...,...
404,FF11_ENX,EURONEXT,ENX,10,False,False
405,FF11_FP,TOTALENERGIES,FP,10,False,False
406,FF11_SEV,SUEZ,SEV,10,False,False
407,FF11_SGO,SAINT-GOBAIN,SGO,10,False,False


In [22]:
get_prefix(compA_companies)

{'1rA', '1rP', 'FF1'}

In [23]:
symbol_paris_compA = set(compA_companies[compA_companies["mid"] == 7]["symbol"])

## CompB - Companies handling

In [24]:
dfs_compB = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "compB"] ["path"]))

100%|██████████| 540/540 [00:02<00:00, 258.03it/s]


In [85]:
for df in dfs_compB:
    df["timestamp"] = df.attrs["timestamp"]
compB_df = pd.concat(dfs_compB)
compB_df = compB_df.reset_index(drop=True)

In [90]:
compB_df.groupby(["symbol", "name"]).first().loc["1rPNACON"]

,last,volume,last_suffix,timestamp
name,,,,
NACON,3.95,4566,c,2020-03-20 09:02:01


In [91]:
compB_df.groupby(["symbol", "name"]).last().loc["1rPNACON"]

,last,volume,last_suffix,timestamp
name,,,,
NACON,1.61,31501,c,2023-12-21 17:02:02


In [25]:
mid = 8 #TODO get mid from db
compB_companies = dfs_to_companie(dfs_compB, mid, is_compA=False)

In [26]:
compB_companies.value_counts("mid")

mid
8     394
10      2
Name: count, dtype: int64

In [27]:
compB_companies

,symbol,name,symbol_nf,mid
0,1rP03227,LATECOERE,03227,8
1,1rP2MX,TERACT R PFD,2MX,8
2,1rP5478,CLARANOVA,5478,8
3,1rPAAA,ALAN ALLMAN ASSOCIATES,AAA,8
4,1rPAAC,ACCOR ACQ COM R PFD,AAC,8
...,...,...,...,...
391,1rPXFAB,X-FAB SILICON,XFAB,8
392,1rPXIL,XILAM ANIMATION,XIL,8
393,1rPYSYT,SALVEPORN01JAN22EX,YSYT,8
394,FF11_AKA,AKKA TECHNOLOGIES,AKA,10


In [28]:
get_prefix(compB_companies)

{'1rP', 'FF1'}

In [29]:
symbol_paris_compB = set(compB_companies[compB_companies["mid"] == 8]["symbol"])

## PEA PME - Companies Handling

In [30]:
dfs_peapme = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "peapme"] ["path"]))

100%|██████████| 324/324 [00:01<00:00, 235.74it/s]


In [84]:
for df in dfs_peapme:
    df["timestamp"] = df.attrs["timestamp"]
peapme_df = pd.concat(dfs_peapme)
peapme_df = peapme_df.reset_index(drop=True)

In [92]:
peapme_df.groupby(["symbol", "name"]).first().loc["1rPNACON"]

,last,volume,last_suffix,timestamp
name,,,,
NACON,7.97,10641,c,2021-01-01 09:02:02


In [93]:
peapme_df.groupby(["symbol", "name"]).last().loc["1rPNACON"]

,last,volume,last_suffix,timestamp
name,,,,
NACON,1.61,31501,c,2023-12-21 17:02:02


In [31]:
mid = 99 #TODO get mid from db
peapme_companies = dfs_to_companie(dfs_peapme, mid, is_compA=False)

In [32]:
peapme_companies

,symbol,name,symbol_nf,mid
0,1rAADUX,ADUX,ADUX,6
1,1rABESI,BESI,BESI,6
2,1rEPALANT,ANTEVENIO,ALANT,8
3,1rEPALAQU,AQUILA,ALAQU,8
4,1rEPALBDM,BD MULTI MEDIA,ALBDM,8
...,...,...,...,...
672,FF11_EVS,EVS BROADCAST EQU,EVS,10
673,FF11_GKTX,GENKYOTEX,GKTX,10
674,FF11_KIN,KINEPOLIS GROUP,KIN,10
675,FF11_MLMAZ,MAZARO,MLMAZ,10


In [33]:
peapme_companies.value_counts("mid")

mid
8     656
10     19
6       2
Name: count, dtype: int64

## View intersection and difference

In [34]:
symbol_paris_peapme = set(peapme_companies[peapme_companies["mid"] == 8]["symbol"])

In [35]:
symbol_paris_comp = symbol_paris_compA | symbol_paris_compB

In [36]:
len(symbol_paris_peapme), len(symbol_paris_comp)

(656, 735)

In [89]:
symbol_paris_peapme.intersection(symbol_paris_compB).difference(symbol_paris_compA)

{'1rP03227',
 '1rP5478',
 '1rPAAA',
 '1rPAB',
 '1rPABCA',
 '1rPABEO',
 '1rPABVX',
 '1rPAELIS',
 '1rPAFME',
 '1rPAFTNV',
 '1rPALBOU',
 '1rPALDLT',
 '1rPALHEX',
 '1rPALLAN',
 '1rPALPM',
 '1rPAMPLI',
 '1rPARVEN',
 '1rPATEME',
 '1rPATI',
 '1rPAXW',
 '1rPBALYO',
 '1rPBELI',
 '1rPBIG',
 '1rPBLC',
 '1rPBSD',
 '1rPCATG',
 '1rPCLA',
 '1rPCOX',
 '1rPCRI',
 '1rPCRINV',
 '1rPDLTA',
 '1rPDVT',
 '1rPECASA',
 '1rPEKI',
 '1rPEPS',
 '1rPERYP',
 '1rPESI',
 '1rPEXA',
 '1rPEXE',
 '1rPEXPL',
 '1rPFDE',
 '1rPFGA',
 '1rPFORSE',
 '1rPFTRN',
 '1rPGAM',
 '1rPGAMNV',
 '1rPGAMT',
 '1rPGBT',
 '1rPGBTNV',
 '1rPGENX',
 '1rPGLO',
 '1rPGNFT',
 '1rPGOE',
 '1rPGRVO',
 '1rPHDF',
 '1rPHEXA',
 '1rPIDIP',
 '1rPINF',
 '1rPIPH',
 '1rPIVA',
 '1rPJBOG',
 '1rPJCQ',
 '1rPKOF',
 '1rPLACR',
 '1rPLAF',
 '1rPLAT',
 '1rPLBIRD',
 '1rPLBON',
 '1rPLHYFE',
 '1rPLIN',
 '1rPLNA',
 '1rPLOCAL',
 '1rPLPE',
 '1rPLTE',
 '1rPMAN',
 '1rPMAU',
 '1rPMBWS',
 '1rPMCPHY',
 '1rPMDW',
 '1rPMEDCL',
 '1rPMETEX',
 '1rPMON',
 '1rPMPI',
 '1rPNACON',
 '1rPNANO

In [61]:
len(symbol_paris_peapme.intersection(symbol_paris_compA)), len(symbol_paris_peapme.intersection(symbol_paris_compB))

(25, 143)

In [38]:
symbol_paris_peapme.difference(symbol_paris_comp)

{'1rEPALANT',
 '1rEPALAQU',
 '1rEPALBDM',
 '1rEPALCAR',
 '1rEPALCLA',
 '1rEPALCLS',
 '1rEPALDEI',
 '1rEPALDLS',
 '1rEPALEO2',
 '1rEPALFRE',
 '1rEPALGEN',
 '1rEPALI2S',
 '1rEPALINS',
 '1rEPALLP',
 '1rEPALMAS',
 '1rEPALMET',
 '1rEPALMIL',
 '1rEPALNEV',
 '1rEPALNXT',
 '1rEPALOBR',
 '1rEPALOCT',
 '1rEPALODI',
 '1rEPALPRO',
 '1rEPALSAS',
 '1rEPALSPO',
 '1rEPALSTA',
 '1rEPALSTW',
 '1rEPALTRA',
 '1rEPALTVO',
 '1rEPALUCR',
 '1rEPALVDM',
 '1rEPALVXM',
 '1rEPALWEB',
 '1rEPALWED',
 '1rEPCBOT',
 '1rP03902',
 '1rP06083',
 '1rP2CRSI',
 '1rP7665',
 '1rPABNX',
 '1rPABVNV',
 '1rPADI',
 '1rPADOC',
 '1rPADONV',
 '1rPADV',
 '1rPADVIC',
 '1rPAL2SI',
 '1rPALA2M',
 '1rPALACT',
 '1rPALADM',
 '1rPALADO',
 '1rPALADV',
 '1rPALAFY',
 '1rPALAGO',
 '1rPALAGP',
 '1rPALAIR',
 '1rPALALO',
 '1rPALAM',
 '1rPALAMG',
 '1rPALANV',
 '1rPALARF',
 '1rPALAST',
 '1rPALATA',
 '1rPALATI',
 '1rPALAUD',
 '1rPALAUR',
 '1rPALAVI',
 '1rPALAVY',
 '1rPALBFR',
 '1rPALBI',
 '1rPALBIO',
 '1rPALBIZ',
 '1rPALBKK',
 '1rPALBLD',
 '1rPALBLU',
 

#TODO
Handle Intersection between compA, compB (Keep the last comp)
Handle Difference between peapme and comp (Keep compA when it is present)

In [39]:
symbol_paris_compA.intersection(symbol_paris_compB)

{'1rPABIO',
 '1rPAKA',
 '1rPARAMI',
 '1rPAREIT',
 '1rPARG',
 '1rPBAIN',
 '1rPBASS',
 '1rPBEN',
 '1rPBLV',
 '1rPBOI',
 '1rPBON',
 '1rPBUR',
 '1rPCBE',
 '1rPCBR',
 '1rPCGG',
 '1rPCNV',
 '1rPDBG',
 '1rPDBV',
 '1rPDEEZR',
 '1rPELIOR',
 '1rPERA',
 '1rPEUCAR',
 '1rPFNAC',
 '1rPIDL',
 '1rPLSS',
 '1rPLSSNV',
 '1rPMDM',
 '1rPMERY',
 '1rPMONC',
 '1rPMTU',
 '1rPNEO',
 '1rPORP',
 '1rPOVH',
 '1rPPHA',
 '1rPRBT',
 '1rPS30',
 '1rPSAVE',
 '1rPSDG',
 '1rPSESL',
 '1rPSMCP',
 '1rPSTF',
 '1rPTCH',
 '1rPTCHCS',
 '1rPTKTT',
 '1rPVETO',
 '1rPVIRP',
 '1rPVK',
 '1rPVLA',
 '1rPVLTSA',
 '1rPWAVE',
 '1rPXFAB'}